In [1]:
import pyxdf
import numpy as np
from sys import argv
from pathlib import Path

In [2]:
xdf_data_path = Path("xdf_data")

In [3]:
xdf_filepaths = list(xdf_data_path.rglob("*.xdf"))

In [4]:
xdf_data = {str(x).split("/")[1] : pyxdf.load_xdf(str(x)) for x in xdf_filepaths}

Stream 3: Calculated effective sampling rate 30.1718 Hz is different from specified rate 90.0000 Hz.
Stream 4: Calculated effective sampling rate 30.1676 Hz is different from specified rate 90.0000 Hz.
Stream 5: Calculated effective sampling rate 30.0871 Hz is different from specified rate 90.0000 Hz.
Stream 5: Calculated effective sampling rate 30.1664 Hz is different from specified rate 90.0000 Hz.
Stream 4: Calculated effective sampling rate 30.1437 Hz is different from specified rate 90.0000 Hz.
Stream 4: Calculated effective sampling rate 30.1845 Hz is different from specified rate 90.0000 Hz.
Stream 4: Calculated effective sampling rate 30.1260 Hz is different from specified rate 90.0000 Hz.
Stream 2: Calculated effective sampling rate 30.1270 Hz is different from specified rate 90.0000 Hz.
Stream 5: Calculated effective sampling rate 30.1097 Hz is different from specified rate 90.0000 Hz.
Stream 3: Calculated effective sampling rate 30.1637 Hz is different from specified rate 90

In [60]:

def separate_trials_expmakers(exp_list):
    for i, string in enumerate(exp_list, start=1):
        if string == "task_block_start|BOX_BLOCK":
            return [exp_list[0:i], exp_list[i:]], False
        elif string == "task_block_start|JEBSEN_TAYLOR":
            return [exp_list[i:], exp_list[0:i]], True
    # If no empty string is found, return the whole list as the first trial, second is empty
    raise Exception("Should not get here separate exp markers!")

def separate_trials_latmarkers(lat_list):
    for i, string in enumerate(lat_list, start=1):
        if string == "repetition_start|2": # Do I need to do a similar logic like above?
            return lat_list[0:i], lat_list[i:]
    # If the marker is not found, return the whole list as first trial, second is empty
    raise Exception("Should not get here separate lat markers!")

In [64]:
def collect_participant_trials(pnum):
    participant_trials = {
        "move_cans": {},
        "move_blocks": {}
    }

    exp_trial_0, exp_trial_1 = None, None
    lat_trial_0, lat_trial_1 = None, None
    trials_swapped = False
    exp_markers_reached = False
    lat_markers_reached = False
    
    data, header = xdf_data[f"sub-{pnum:03}"]  # Assuming xdf_data is already loaded
    for stream in data:
        stream_series = stream["time_series"]
        stream_name = stream["info"]["name"][0]
        stream_series_list = list(stream_series)

        print(stream_name)
        
        if stream_name == "LatencyMarkers":
            if lat_markers_reached:
                raise Exception("Duplication of latency markers!")
            lat_markers_reached = True
            
            stream_series_np = np.array(stream_series).reshape(len(stream_series))
            lat_trial_0, lat_trial_1 = separate_trials_latmarkers(list(stream_series_np))
            if exp_markers_reached:
                if trials_swapped:
                    participant_trials["move_cans"]["exp_markers"] = exp_trial_1
                    participant_trials["move_blocks"]["exp_markers"] = exp_trial_0
                    participant_trials["move_cans"]["lat_markers"] = lat_trial_1
                    participant_trials["move_blocks"]["lat_markers"] = lat_trial_0
                else:
                    participant_trials["move_cans"]["exp_markers"] = exp_trial_0
                    participant_trials["move_blocks"]["exp_markers"] = exp_trial_1
                    participant_trials["move_cans"]["lat_markers"] = lat_trial_0
                    participant_trials["move_blocks"]["lat_markers"] = lat_trial_1
            
        elif stream_name == "ExpMarkers":
            if exp_markers_reached:
                raise Exception("Duplication of experiment markers!")
            exp_markers_reached = True
            
            stream_series_np = np.array(stream_series).reshape(len(stream_series))
            # Not swapped can trial and then block trial
            [exp_trial_0, exp_trial_1], trials_swapped = separate_trials_expmakers(list(stream_series_np))

            if lat_markers_reached:
                if trials_swapped:
                    participant_trials["move_cans"]["exp_markers"] = exp_trial_1
                    participant_trials["move_blocks"]["exp_markers"] = exp_trial_0
                    participant_trials["move_cans"]["lat_markers"] = lat_trial_1
                    participant_trials["move_blocks"]["lat_markers"] = lat_trial_0
                else:
                    participant_trials["move_cans"]["exp_markers"] = exp_trial_0
                    participant_trials["move_blocks"]["exp_markers"] = exp_trial_1
                    participant_trials["move_cans"]["lat_markers"] = lat_trial_0
                    participant_trials["move_blocks"]["lat_markers"] = lat_trial_1

    if not lat_markers_reached or not exp_markers_reached:
        raise Exception("Some of the required markers were not reached!")
    
    return participant_trials

In [76]:
participant_data = collect_participant_trials(1)

LatencyMarkers
ExpMarkers
ExpMarkers


Exception: Duplication of experiment markers!

In [66]:
for key in participant_data.keys():
    print(f"{key} with values {participant_data[key].keys()}")

move_cans with values dict_keys(['exp_markers', 'lat_markers'])
move_blocks with values dict_keys(['exp_markers', 'lat_markers'])
